In [1]:
from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

In [2]:
gauss = io.load('data/gauss4.p')

In [3]:
list(gauss.pts.values.columns)

['z']

In [4]:
list(gauss.pts.x.columns)

['x', 'y']